<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-01 10:31:18
-------------------
qualified stocks: 95
with latest results: 30
still star stocks: 20
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  4.51 L
Current:  1.37 C
-------------------
Today PnL: 9.82 K (0.07%)
Current PnL: -17.58 L (-11.96%)
CY Booked + Current PnL: -9.55 L (-6.49%)
-------------------
Total profit:  3.24 L
Total loss:  -20.82 L
-------------------
Total Booked + Current PnL: 17.43 L (14.52%)
Total Booked PnL: 35.01 L (29.16%)
Curr Year Booked PnL: 8.03 L (5.86%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 75.80 L (55.31%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,BRITANNIA,5190.70,6446.05,17.58,H-LC,9.76,105543.0,12110.0,10491.0,1.60,12.96,9.94,24.18,31.0,1.15,0.80,28.16,XY25,ATH,FMCG
4,APOLLOHOSP,6904.43,8285.00,-18.07,H-LC,6.48,111615.0,8049.0,12657.0,-0.76,7.77,11.34,20.00,15.0,0.64,0.84,22.94,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-21.55,H-LC,5.22,107792.0,3480.0,14056.0,-1.37,3.34,13.04,16.81,10.0,0.25,0.81,19.92,X5K,ATH,METALS
29,DIXON,14367.00,18940.15,-25.71,H-MC,6.15,136768.0,21832.0,14757.0,1.51,18.99,10.79,31.83,85.0,1.48,1.03,53.92,X40N,ATH,IT
74,SIS,477.00,477.00,2350.26,H-SC,6.84,65926.0,-15164.0,15163.0,0.36,-18.70,23.00,0.00,157.0,-1.00,0.50,33.61,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.99,M-SC,41.76,64583.0,-423.0,151350.0,0.10,-0.65,234.35,232.18,199.0,-0.00,0.49,0.34,XY24,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,9.28,H-SC,19.98,101729.0,766.0,93764.0,0.75,0.76,92.17,93.63,149.0,0.01,0.77,51.52,SR,ATH,CHEMICALS
20,CAMS,3643.00,5226.82,-8.36,H-SC,0.80,103001.0,997.0,43353.0,-1.56,0.98,42.09,43.48,120.0,0.02,0.78,17.92,X40N,ATH,MISC
38,HEROMOTOCO,4311.81,5949.07,-7.14,H-MC,3.63,149716.0,-1197.0,58494.0,0.40,-0.79,39.07,37.97,90.0,-0.02,1.13,24.31,AR,ATH,AUTO
94,WIPRO,243.46,420.00,-11.48,M-LC,5.18,151962.0,1017.0,108440.0,-1.29,0.67,71.36,72.51,55.0,0.01,1.15,6.74,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-21.14,H-LC,1.74,207391.0,2891.0,38533.0,-2.62,1.41,18.58,20.25,9.0,0.08,1.56,10.27,X40N,BTT,PHARMA
56,LTIM,5564.16,7230.2,-5.37,H-LC,2.82,182124.0,-18186.0,78168.0,-0.92,-9.08,42.92,29.94,24.0,-0.23,1.37,25.82,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.21,H-LC,2.85,186386.0,-14120.0,62887.0,-0.19,-7.04,33.74,24.33,30.0,-0.22,1.41,7.48,X40N,BTT,FINANCE
5,ASIANPAINT,2961.56,4250.0,-15.85,H-LC,3.17,167212.0,-34174.0,121780.0,2.63,-16.97,72.83,43.51,19.0,-0.28,1.26,16.40,X40,BTT,PAINTS
60,NESTLEIND,2268.60,2755.0,-16.02,H-LC,4.51,266432.0,1006.0,55897.0,1.31,0.38,20.98,21.44,26.0,0.02,2.01,7.05,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,DEN,52.79,75.0,122.80,M-SC,3.37,62336.0,-22128.0,57667.0,5.27,-26.20,92.51,42.07,238.0,-0.38,0.47,32.38,AR,NTT,ENTERTAINMENT
35,HATHWAY,19.74,31.4,972.46,H-SC,2.37,61600.0,-17360.0,64002.0,2.80,-21.99,103.90,59.07,159.0,-0.27,0.46,23.60,XR,NTT,ENTERTAINMENT
14,BERGEPAINT,561.33,680.0,-16.88,H-MC,10.25,235102.0,7763.0,40296.0,2.76,3.41,17.14,21.14,107.0,0.19,1.77,31.63,XY24,NTT,PAINTS
5,ASIANPAINT,2961.56,4250.0,-15.85,H-LC,3.17,167212.0,-34174.0,121780.0,2.63,-16.97,72.83,43.51,19.0,-0.28,1.26,16.40,X40,BTT,PAINTS
40,HINDUNILVR,2413.81,2723.0,-7.63,H-LC,4.80,289475.0,19128.0,15487.0,2.51,7.08,5.35,12.81,28.0,1.24,2.18,21.06,XY25,NTT,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-21.14,H-LC,1.74,207391.0,2891.0,38533.0,-2.62,1.41,18.58,20.25,9.0,0.08,1.56,10.27,X40N,BTT,PHARMA
90,VALIANTORG,512.64,838.00,-310.85,H-SC,20.44,89700.0,-43586.0,128181.0,-2.45,-32.70,142.90,63.47,147.0,-0.34,0.68,50.36,XR,NTT,CHEMICALS
59,NATIONALUM,189.63,247.44,-52.42,H-MC,3.01,96179.0,-4325.0,34961.0,-1.94,-4.30,36.35,30.49,81.0,-0.12,0.73,26.71,MH,ATH,METALS
81,TATAMOTORS,761.77,1065.00,-45.25,M-LC,4.77,237711.0,-38812.0,148878.0,-1.67,-14.04,62.63,39.81,54.0,-0.26,1.79,13.92,XY24,NTT,AUTO
75,SONACOMS,578.05,1006.00,-39.99,M-SC,13.91,77394.0,-23765.0,98654.0,-1.62,-23.49,127.47,74.03,202.0,-0.24,0.58,9.13,AR,BTT,AUTO


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SYMPHONY,1306.42,1306.0,-30.13,M-SC,8.83,148882.0,-22259.0,22198.0,1.94,-13.01,14.91,-0.03,197.0,-1.00,1.12,7.74,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,129.91,H-SC,5.77,150598.0,-23961.0,23990.0,1.21,-13.73,15.93,0.02,135.0,-1.00,1.14,30.59,OX40N,NTT,PAINTS
86,UJJIVANSFB,52.77,53.0,36.52,H-SC,20.83,118800.0,-23679.0,24295.0,-0.61,-16.62,20.45,0.44,163.0,-0.97,0.90,40.22,OX40N,NTT,BANKS
52,KANSAINER,299.63,340.0,-68.02,H-SC,1.70,223011.0,-46656.0,82982.0,1.18,-17.30,37.21,13.47,136.0,-0.56,1.68,12.80,XY24,NTT,PAINTS
80,TATAELXSI,7332.28,7332.0,-19.16,H-MC,1.92,84826.0,-17826.0,17822.0,-0.57,-17.37,21.01,-0.00,101.0,-1.00,0.64,28.95,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TCS,3794.03,4998.00,-27.70,H-LC,13.85,235591.0,-60343.0,154241.0,-0.54,-20.39,65.47,31.73,1.0,-0.39,1.78,0.00,X40,BTT,IT
32,ENRIN,1377.95,3237.90,NaN,NaN,3.16,79960.0,45511.0,984.0,-1.22,132.11,1.23,134.98,2.0,46.25,0.60,22.27,AR,ATH,ELECTRICAL
91,VBL,492.64,671.64,-8.52,H-LC,11.74,277500.0,12460.0,83833.0,-1.21,4.70,30.21,36.33,3.0,0.15,2.09,18.63,X40N,ATH,FMCG
48,INFY,1461.46,2275.00,-16.73,H-LC,7.45,267336.0,5735.0,139897.0,-1.03,2.19,52.33,55.67,4.0,0.04,2.02,8.43,X40,BTT,IT
1,ABB,5551.76,7934.00,-40.16,H-LC,7.59,213544.0,-2975.0,95881.0,-0.63,-1.37,44.90,42.91,5.0,-0.03,1.61,11.62,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,4.26,50429.0,9174.0,8260.0,-1.00,22.24,16.38,42.26,198.0,1.11,0.38,18.73,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,20.01,H-SC,12.50,153793.0,27593.0,18578.0,1.11,21.86,12.08,36.59,156.0,1.49,1.16,37.12,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3237.90,NaN,NaN,3.16,79960.0,45511.0,984.0,-1.22,132.11,1.23,134.98,2.0,46.25,0.60,22.27,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,131922.0,6625.0,52452.0,2.10,5.29,39.76,47.15,144.0,0.13,1.00,65.04,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,20.01,H-SC,12.50,153793.0,27593.0,18578.0,1.11,21.86,12.08,36.59,156.0,1.49,1.16,37.12,XY24,NTT,HEALTHCARE
3,ALKYLAMINE,2347.98,4546.37,9.28,H-SC,19.98,101729.0,766.0,93764.0,0.75,0.76,92.17,93.63,149.0,0.01,0.77,51.52,SR,ATH,CHEMICALS
26,DABUR,505.20,735.00,-2.87,H-MC,5.17,210249.0,13221.0,76404.0,1.91,6.71,36.34,45.49,100.0,0.17,1.59,20.63,XY24,BTT,FMCG
15,BLUESTARCO,1646.70,2326.38,-7.11,H-SC,9.47,174120.0,9450.0,58522.0,0.18,5.74,33.61,41.28,119.0,0.16,1.31,14.18,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3237.90,NaN,NaN,3.16,79960.0,45511.0,984.0,-1.22,132.11,1.23,134.98,2.0,46.25,0.60,22.27,AR,ATH,ELECTRICAL
53,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,131922.0,6625.0,52452.0,2.10,5.29,39.76,47.15,144.0,0.13,1.00,65.04,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,13.35,137270.0,13934.0,119823.0,0.03,11.30,87.29,108.45,118.0,0.12,1.04,36.18,AR,ATH,MISC
93,WHIRLPOOL,1219.98,2270.00,-41.08,M-SC,7.05,99180.0,7682.0,71072.0,-0.91,8.40,71.66,86.07,211.0,0.11,0.75,43.47,XR,NTT,DURABLES
94,WIPRO,243.46,420.00,-11.48,M-LC,5.18,151962.0,1017.0,108440.0,-1.29,0.67,71.36,72.51,55.0,0.01,1.15,6.74,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,7.33,19338.0,-30255.0,1456.0,0.82,-61.01,7.53,-58.07,261.0,-20.78,0.15,29.68,XY24,ATH,MISC
74,SIS,477.00,477.00,2350.26,H-SC,6.84,65926.0,-15164.0,15163.0,0.36,-18.70,23.00,0.00,157.0,-1.00,0.50,33.61,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.14,68314.0,-45235.0,85290.0,-0.47,-39.84,124.85,35.27,267.0,-0.53,0.52,67.20,XR,NTT,HOTELS
32,ENRIN,1377.95,3237.90,NaN,NaN,3.16,79960.0,45511.0,984.0,-1.22,132.11,1.23,134.98,2.0,46.25,0.60,22.27,AR,ATH,ELECTRICAL
57,MASFIN,326.60,399.50,-19.60,H-SC,16.73,91455.0,-6525.0,28397.0,-1.04,-6.66,31.05,22.32,151.0,-0.23,0.69,32.64,XR,ATH,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,7.33,19338.0,-30255.0,1456.0,0.82,-61.01,7.53,-58.07,261.0,-20.78,0.15,29.68,XY24,ATH,MISC
28,DIGITIDE,188.38,267.99,NaN,NaN,4.26,50429.0,9174.0,8260.0,-1.00,22.24,16.38,42.26,198.0,1.11,0.38,18.73,XY24,ATH,IT
29,DIXON,14367.00,18940.15,-25.71,H-MC,6.15,136768.0,21832.0,14757.0,1.51,18.99,10.79,31.83,85.0,1.48,1.03,53.92,X40N,ATH,IT
72,SHALBY,261.39,327.00,982.93,M-SC,8.96,147262.0,-33881.0,79345.0,1.36,-18.70,53.88,25.10,233.0,-0.43,1.11,18.02,XY24,NTT,HEALTHCARE
58,MEDANTA,1087.93,1486.00,20.01,H-SC,12.50,153793.0,27593.0,18578.0,1.11,21.86,12.08,36.59,156.0,1.49,1.16,37.12,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.81
1,25,41.68
2,50,70.76


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.20
LC    35.50
MC    22.21
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.80
X40N     14.85
X40      12.63
XY25     11.04
XR        9.02
AR        8.30
OX40N     7.90
X5K       2.19
MH        1.73
X200      1.37
SR        1.21
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.53
H-LC    27.42
H-MC    19.28
M-SC    12.23
M-LC     7.03
M-MC     2.59
L-SC     1.44
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.79,0.18,33.19
IT,11.74,-16.86,74.44
FINANCE,8.43,-16.90,59.31
BANKS,7.45,-15.50,62.64
PAINTS,5.85,-12.50,34.67
ELECTRICAL,5.41,-2.76,51.66
MISC,5.35,-22.42,85.87
HEALTHCARE,4.72,3.62,21.98
AUTO,4.30,-18.53,70.09


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2767478.0,25
XR,1098713.0,13
AR,1046780.0,10
X40,781388.0,10
X40N,580357.0,12
XY25,476771.0,8
OX40N,330966.0,10
SR,182995.0,2
X5K,149080.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2676904.0,30
M-SC,1350617.0,15
H-LC,1230824.0,20
H-MC,1151005.0,15
M-LC,446893.0,5
M-MC,334740.0,2
L-SC,270150.0,3
L-MC,57493.0,1
L-LC,50783.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      923642.0      7
M-SC       XY24      831874.0      7
H-SC       AR        611231.0      4
           XR        512410.0      6
H-LC       X40       510179.0      6
H-MC       XY24      440065.0      4
M-MC       XY24      334740.0      2
H-LC       X40N      285961.0      6
H-SC       X40N      220848.0      4
H-LC       AR        219750.0      2
H-MC       XY25      207962.0      2
           X40       196036.0      3
M-LC       XY24      186606.0      2
H-SC       SR        182995.0      2
L-SC       XR        182449.0      2
H-SC       OX40N     173326.0      6
M-SC       AR        156321.0      2
M-LC       X5K       135024.0      1
H-MC       XR        122117.0      1
M-SC       XY25      119328.0      1
           XR        115804.0      2
M-LC       XR        108440.0      1
L-SC       OX40N      87701.0      1
H-LC       XY25       81875.0      3
           X200       78168.0      1
M-SC       X40        75173.0      1
H-MC       X40N       73548.0      2
           AR         58494.0      1
L-MC       XR         57493.0      1
H-SC       MH         52452.0      1
M-SC       OX40N      52117.0      2
L-LC       XY25       50783.0      1
H-LC       XY24       40835.0      1
H-MC       MH         34961.0      1
           OX40N      17822.0      1
M-LC       XY25       16823.0      1
H-LC       X5K        14056.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
